<a href="https://colab.research.google.com/github/russomanno-phil/blank-app/blob/main/SHAK_Working_File.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas requests

import pandas as pd
import requests


In [ ]:
# I am loading data about poppulations from the census API
# --- ACS 2023 state-level population + median income ---

BASE_ACS = "https://api.census.gov/data/2023/acs/acs1"

acs_vars = ["NAME", "B01003_001E", "B19013_001E"]  # state name, population, median income

acs_params = {
    "get": ",".join(acs_vars),
    "for": "state:*"
}

resp_acs = requests.get(BASE_ACS, params=acs_params)
resp_acs.raise_for_status()

data_acs = resp_acs.json()
df_acs = pd.DataFrame(data_acs[1:], columns=data_acs[0])

# Clean / rename
# Rename the FIPS 'state' column first to avoid conflict, then rename others
df_acs = df_acs.rename(columns={df_acs.columns[-1]: "state_fips"})
df_acs = df_acs.rename(columns={
    "NAME": "state",
    "B01003_001E": "population",
    "B19013_001E": "median_income"
})

df_acs["population"] = df_acs["population"].astype(int)
df_acs["median_income"] = df_acs["median_income"].astype(int)

# --- Add land area + pop density ---
area_url = "https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv"
area_df = pd.read_csv(area_url)  # columns: state, area (sq. mi)

df_acs = df_acs.merge(area_df, on="state", how="left")
df_acs["pop_density"] = df_acs["population"] / df_acs["area (sq. mi)"]

# --- Add state abbreviations ---
abbr_url = "https://raw.githubusercontent.com/jasonong/List-of-US-States/master/states.csv"
abbr_df = pd.read_csv(abbr_url)  # columns: state, Abbreviation

df_acs = df_acs.merge(abbr_df, left_on="state", right_on="State", how="left")
df_acs = df_acs.rename(columns={"Abbreviation": "state_abbrev"})

df_acs.head()

,state,population,median_income,state_fips,area (sq. mi),pop_density,State,state_abbrev
0,Alabama,5108468,62212,01,52423,97.447075,Alabama,AL
1,Alaska,733406,86631,02,656425,1.117273,Alaska,AK
2,Arizona,7431344,77315,04,114006,65.183797,Arizona,AZ
3,Arkansas,3067732,58700,05,53182,57.683652,Arkansas,AR
4,California,38965193,95521,06,163707,238.017880,California,CA


In [ ]:
# --- CBP 2022: limited-service restaurants (NAICS 7225) by state ---

BASE_CBP = "https://api.census.gov/data/2022/cbp"

cbp_params = {
    "get": "ESTAB,NAME,NAICS2017",
    "for": "state:*",
    "NAICS2017": "7225",
}

resp_cbp = requests.get(BASE_CBP, params=cbp_params)
resp_cbp.raise_for_status()

data_cbp = resp_cbp.json()
df_cbp = pd.DataFrame(data_cbp[1:], columns=data_cbp[0])

# Rename the FIPS 'state' column first to avoid conflict with 'NAME' column
df_cbp = df_cbp.rename(columns={df_cbp.columns[-1]: "state_fips_cbp"})

df_cbp = df_cbp.rename(columns={
    "NAME": "state",
    "ESTAB": "qsr_establishments"
})
df_cbp["qsr_establishments"] = df_cbp["qsr_establishments"].astype(int)

df_cbp = df_cbp[["state", "qsr_establishments"]]

df_cbp.head()

,state,qsr_establishments
0,Alabama,8008
1,Alaska,1358
2,Arizona,11537
3,Arkansas,4910
4,California,78535


In [ ]:
# I uploaded the amount of stores from the 10k by what state they are located in
# --- Load SHAK state file you uploaded ---
shak = pd.read_csv("shak_state_locations_2024.csv")

# Rename 'State' to 'state' in shak to match df_acs
shak = shak.rename(columns={"State": "state"})

# Rename 'Total' to 'current_company_shacks' for later calculations
shak = shak.rename(columns={"Total": "current_company_shacks"})

# Merge shak with abbr_df to get 'state_abbrev'
# abbr_df was created in cell _qg47Kiy4uyT and has columns 'State' and 'Abbreviation'
shak = shak.merge(abbr_df, left_on="state", right_on="State", how="left")
shak = shak.rename(columns={"Abbreviation": "state_abbrev"})
# Drop the redundant 'State' column from abbr_df that was added during the merge
shak = shak.drop(columns=["State"]) # Drop 'State' as it's the column from abbr_df, not 'State_y'

# Check shapes
print("SHAK:", shak.shape)
print("ACS:", df_acs.shape)
print("CBP:", df_cbp.shape)

# --- Merge SHAK + ACS ---
# Now shak has 'state' and 'state_abbrev' for a successful merge
df = shak.merge(df_acs, on=["state", "state_abbrev"], how="inner")

# --- Merge QSR (competition) ---
df = df.merge(df_cbp, on="state", how="left")

# --- Competition per 100k residents ---
df["qsr_per_100k"] = df["qsr_establishments"] / (df["population"] / 100_000)

# --- Core penetration metric ---
df["shacks_per_million"] = df["current_company_shacks"] / (df["population"] / 1_000_000)

df.head()

SHAK: (35, 5)
ACS: (52, 8)
CBP: (56, 2)


,state,Company Operated,Licensed,current_company_shacks,state_abbrev,population,median_income,state_fips,area (sq. mi),pop_density,State,qsr_establishments,qsr_per_100k,shacks_per_million
0,Alabama,1,0,1,AL,5108468,62212,01,52423,97.447075,Alabama,8008,156.759326,0.195753
1,Arizona,5,1,6,AZ,7431344,77315,04,114006,65.183797,Arizona,11537,155.247826,0.807391
2,California,49,1,50,CA,38965193,95521,06,163707,238.017880,California,78535,201.551677,1.283197
3,Colorado,9,1,10,CO,5877610,92911,08,104100,56.461191,Colorado,11409,194.109510,1.701372
4,Connecticut,8,0,8,CT,3617176,91665,09,5544,652.448773,Connecticut,7472,206.569987,2.211670


In [ ]:
#How to get to 1500 units
# --- Choose "mature" benchmark states for density ---
mature_states = ["New York", "New Jersey"]  # tweak if needed

mature = df[df["state"].isin(mature_states)]
mature_density = mature["shacks_per_million"].mean()

# Conservative target_density: 70% of mature
target_density = 0.7 * mature_density
print("Mature density:", mature_density, "Target density:", target_density)

# --- Baseline potential units based on target density ---
df["potential_shacks_density"] = (df["population"] / 1_000_000) * target_density

# --- Build a demand score: income + density - QSR competition ---

# Normalize features
df["income_z"] = (df["median_income"] - df["median_income"].mean()) / df["median_income"].std()
df["density_z"] = (df["pop_density"] - df["pop_density"].mean()) / df["pop_density"].std()
df["qsr_z"] = (df["qsr_per_100k"] - df["qsr_per_100k"].mean()) / df["qsr_per_100k"].std()

# Demand score: higher income & density are good, higher QSR density is bad
df["demand_score"] = (
    0.5 * df["income_z"] +
    0.3 * df["density_z"] -
    0.2 * df["qsr_z"]
)

# Convert demand_score into multiplier (~0.7x to 1.3x)
df["demand_multiplier"] = 1 + 0.15 * df["demand_score"].clip(-2, 2)

# Final potential at maturity
df["potential_shacks"] = df["potential_shacks_density"] * df["demand_multiplier"]


Mature density: 2.8298999490366965 Target density: 1.9809299643256875


In [ ]:
# --- White space = potential - current ---
df["white_space"] = (df["potential_shacks"] - df["current_company_shacks"]).clip(lower=0)

current_total = int(df["current_company_shacks"].sum())
target_total = 1500
incremental_needed = max(target_total - current_total, 0)

print("Current total company-operated:", current_total)
print("Incremental needed:", incremental_needed)

if incremental_needed > 0 and df["white_space"].sum() > 0:
    df["white_space_share"] = df["white_space"] / df["white_space"].sum()
    df["recommended_adds"] = (df["white_space_share"] * incremental_needed).round()
else:
    df["white_space_share"] = 0
    df["recommended_adds"] = 0

df["recommended_total"] = df["current_company_shacks"] + df["recommended_adds"]

# Sort for sanity check
df = df.sort_values("recommended_adds", ascending=False)

df.head(15)


Current total company-operated: 373
Incremental needed: 1127


,state,Company Operated,Licensed,current_company_shacks,state_abbrev,population,median_income,state_fips,area (sq. mi),pop_density,...,income_z,density_z,qsr_z,demand_score,demand_multiplier,potential_shacks,white_space,white_space_share,recommended_adds,recommended_total
2,California,49,1,50,CA,38965193,95521,06,163707,238.017880,...,1.146869,-0.173405,0.590357,0.403342,1.060501,81.857250,31.857250,0.125899,142.0,192.0
29,Texas,26,2,28,TX,30503301,75780,48,268601,113.563617,...,-0.389120,-0.247162,-0.552417,-0.158225,0.976266,58.990793,30.990793,0.122475,138.0,166.0
7,Florida,22,2,24,FL,22610726,73311,12,65758,343.847532,...,-0.581226,-0.110686,-0.531122,-0.217594,0.967361,43.328349,19.328349,0.076385,86.0,110.0
24,Ohio,7,1,8,OH,11785935,67769,39,44828,262.914585,...,-1.012433,-0.158650,-0.399609,-0.473889,0.928917,21.687520,13.687520,0.054093,61.0,69.0
8,Georgia,7,1,8,GA,11029227,74632,13,59441,185.549150,...,-0.478443,-0.204500,-0.232344,-0.254103,0.961885,21.015377,13.015377,0.051437,58.0,66.0
9,Illinois,12,0,12,IL,12549689,80306,17,57918,216.680289,...,-0.036965,-0.186050,-0.032072,-0.067883,0.989817,24.606917,12.606917,0.049822,56.0,68.0
23,North Carolina,7,2,9,NC,10835491,70804,37,53821,201.324594,...,-0.776288,-0.195151,-0.340243,-0.378641,0.943204,20.245257,11.245257,0.044441,50.0,59.0
16,Michigan,8,0,8,MI,10037261,69183,26,96810,103.680002,...,-0.902413,-0.253019,-0.634212,-0.400270,0.939959,18.689319,10.689319,0.042244,48.0,56.0
32,Washington,6,0,6,WA,7812880,94605,53,71303,109.572949,...,1.075598,-0.249527,0.184951,0.425951,1.063893,16.465619,10.465619,0.041360,47.0,53.0
1,Arizona,5,1,6,AZ,7431344,77315,04,114006,65.183797,...,-0.269686,-0.275834,-0.991775,-0.019238,0.997114,14.678491,8.678491,0.034297,39.0,45.0
